In [1]:
import random, os
import numpy as np
import torch

def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(42)

# Load data

In [2]:
import datasets
import pandas as pd
from ast import literal_eval
df_dir = '../input/vietnews-dataset/test.csv'

In [3]:
import pandas as pd
df = pd.read_csv(df_dir)
df

,Filename,Title,Abstract,Content,Keyword
0,000001.txt.seg,Bản_án cho đối_tượng giả_danh công_an để lừa_đảo,"Ngày 25/2 , TAND TP. Đà_Nẵng tuyên_phạt Hồ_Xuâ...","Theo lời khai của Huy tại phiên_toà , để có ti...",Huy tại phiên_toà .
1,000002.txt.seg,Nam thanh_niên thủ_dâm trên xe_buýt từng bị xử...,Phát_hiện nam thanh_niên đang thủ_dâm trên xe_...,"Tối 27/6 báo Người Lao_Động đưa tin , một lãnh...",Nam thanh_niên ( áo đen ) đã có hành_động thủ_...
2,000003.txt.seg,"Không được công_nhận , thuốc "" sinh con theo ý...",“ Baby_Support ” và “ Hello baby ” là hai cái ...,Quảng_cáo như thần_dược\nTrên diễn_đàn của các...,Hai loại thuốc được quảng_cáo có_thể hỗ_trợ si...
3,000004.txt.seg,VEC từ_chối phục_vụ vĩnh_viễn 2 ô_tô gây_rối t...,Liên_quan đến vụ gây_rối tại trạm thu phí trên...,"Như báo Người Đưa_Tin đã phản_ánh trước đó , T...",Trạm thu phí Long_Phước trên cao_tốc TP. HCM -...
4,000005.txt.seg,VKSND tỉnh Sóc_Trăng xin_lỗi oan sai “ nữ_hoàn...,Bà Huỳnh_Ngọc_Bích được mọi người biết đến là ...,"Sáng 28/5 , VKSND tỉnh Sóc_Trăng tổ_chức buổi ...","Ông Nguyễn_Hồng_Phuông , Phó viện Trưởng VKSND..."
...,...,...,...,...,...
22639,022640.txt.seg,Mưa_đá trút xuống vùng_biên Nghệ_An làm hỏng n...,Hạt đá với đường_kính hơn 1 cm trút liên_tục n...,Mưa_đá chiều 6/4 tại xã Na_Ngoi ( Kỳ_Sơn ) .\n...,Mưa_đá chiều 6/4 tại xã Na_Ngoi ( Kỳ_Sơn ) .\n...
22640,022641.txt.seg,Tàu_cánh_ngầm TP HCM - Vũng_Tàu được chạy trở_lại,Sau gần một năm tạm cấm tất_cả các tàu_cánh_ng...,""" Phải đảm_bảo an_toàn tuyệt_đối cho hành_khác...",Sau vụ tàu_cánh_ngầm Vina_Express bị cháy trên...
22641,022642.txt.seg,TP HCM nghiên_cứu chính_sách miễn học_phí THPT,"Trong khi chờ cơ_chế miễn học_phí cấp THCS , t...","Ngày 19/11 , đến thăm một_số nhà_giáo lão_thàn...",Học_sinh trường THPT Gia_Định ( quận Bình_Thạn...
22642,022643.txt.seg,Thứ_trưởng Trần_Văn_Tùng : ' Tạo mọi thuận_lợi...,Khẳng_định kiều_bào là nguồn_lực khoa_học - cô...,"Ngày 9/2 , gặp_gỡ các chuyên_gia trí_thức kiều...",Thứ_trưởng Trần_Văn_Tùng nói_chuyện với kiều_b...


In [4]:

df['Content'] = df['Content'].fillna('').str.replace('_', ' ')
df['Abstract'] = df['Abstract'].fillna('').str.replace('_', ' ')
test_df = pd.DataFrame({
    'text': df['Content'],
    'abstract': df['Abstract']
})

In [5]:
test_df

,text,abstract
0,"Theo lời khai của Huy tại phiên toà , để có ti...","Ngày 25/2 , TAND TP. Đà Nẵng tuyên phạt Hồ Xuâ..."
1,"Tối 27/6 báo Người Lao Động đưa tin , một lãnh...",Phát hiện nam thanh niên đang thủ dâm trên xe ...
2,Quảng cáo như thần dược\nTrên diễn đàn của các...,“ Baby Support ” và “ Hello baby ” là hai cái ...
3,"Như báo Người Đưa Tin đã phản ánh trước đó , T...",Liên quan đến vụ gây rối tại trạm thu phí trên...
4,"Sáng 28/5 , VKSND tỉnh Sóc Trăng tổ chức buổi ...",Bà Huỳnh Ngọc Bích được mọi người biết đến là ...
...,...,...
22639,Mưa đá chiều 6/4 tại xã Na Ngoi ( Kỳ Sơn ) .\n...,Hạt đá với đường kính hơn 1 cm trút liên tục n...
22640,""" Phải đảm bảo an toàn tuyệt đối cho hành khác...",Sau gần một năm tạm cấm tất cả các tàu cánh ng...
22641,"Ngày 19/11 , đến thăm một số nhà giáo lão thàn...","Trong khi chờ cơ chế miễn học phí cấp THCS , t..."
22642,"Ngày 9/2 , gặp gỡ các chuyên gia trí thức kiều...",Khẳng định kiều bào là nguồn lực khoa học - cô...


In [6]:
test_df.shape

(22644, 2)

# Test

In [7]:
!pip install evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.9 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6bd0e17287d33ca6a08ec78692e6db7d01ee759cc41be81c3de8247677d4f2b5
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is 

In [8]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## load model

In [9]:
from transformers import pipeline
model_path = 'VietAI/vit5-base-vietnews-summarization'
summarizer = pipeline(
    "summarization",
    model=model_path,
    tokenizer=model_path
)
def summarize_text(text: str, max_length=256) -> str:
    """
    Tóm tắt văn bản tiếng Việt.
    """
    text = "vietnews: " + text + " </s>"
    res = summarizer(
        text,
        max_length=max_length,
        early_stopping=True,
    )[0]['summary_text']
    return res

def test(df):
    texts = df['text'].tolist()
    predictions = []
    pred_labels = []

    print(f"Đang chạy trên {len(texts)} văn bản...")

    for text in tqdm(texts):
        result = summarize_text(text)
        predictions.append(result)

    return df['abstract'], predictions



2025-07-05 08:40:21.042991: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751704821.228042      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751704821.282926      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/904M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [10]:
import torch
from torch import nn
from tqdm.notebook import tqdm

import evaluate


def eval(true_abstract, predicts):
    rouge = evaluate.load("rouge")

    scores = rouge.compute(
        predictions=predicts,
        references=true_abstract,
        use_stemmer=True
    )

    # Hiển thị kết quả
    for key in ["rouge1", "rouge2", "rougeL"]:
        print(f"{key}: {scores[key]:.4f}") 

    


In [11]:
true, predicts = test(test_df)

Đang chạy trên 22644 văn bản...


  0%|          | 0/22644 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Your max_length is set to 256, but your input_length is only 228. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=114)
Your max_length is set to 256, but your input_length is only 165. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=82)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Your max_length is set to 256, but your input_length is only 247. Since this is a summ

In [12]:
test_cm = eval(true, predicts)
#test_cm = eval(test_df['abstract'], test_df['abstract'])

rouge1: 0.6258
rouge2: 0.3316
rougeL: 0.4278
